# Build datamaps for all models

In [1]:
from lcls_live.datamaps.tabular import TabularDataMap, datamap_from_tao_data
from lcls_live.datamaps.klystron import KlystronDataMap, klystron_pvinfo, existing_LCLS_klystrons_sector_station, subbooster_pvinfo, SUBBOOSTER_SECTORS

from scipy.constants import e as e_charge
from pytao import Tao
import json
import os
from lcls_live import data_dir
import pandas as pd

# cu_hxr and cu_sxr

In [2]:
from lcls_live.datamaps.builders import *

In [3]:
MASTER = {} # Master model datamap dict

## Utility function for creating map for given model 

In [4]:
def build_json_rep(model):
    #set up tao
    # Basic model with options
    MODEL = model
    OPTIONS = f'-noplot'
    INIT = f'-init $LCLS_LATTICE/bmad/models/{MODEL}/tao.init {OPTIONS}'
    tao = Tao(INIT)
    
    
    rep = []
    
    
    dm = build_bpm_dm(tao, model)
    rep.append(
        {
        "name": "bpms",
        "class": "tabular",
        "data": dm.to_json()
        }
    )    
    
    # Correctors
    dm = build_corrector_dm(tao)
    rep.append(
        {
        "name": "correctors",
        "class": "tabular",
        "data": dm.to_json()
        }
    )    
    
    
    #subbooster_dm
    dm = build_subbooster_dm(model)
    rep.append(
        {
        "name": "subboosters",
        "class": "tabular",
        "data": dm.to_json()
        }
    )
    
    #linac
    dm = build_linac_dm(model)
    rep.append(
        {
        "name": "linac",
        "class": "tabular",
        "data": dm.to_json()
        }
    )
    
    #klystron
    dms = build_klystron_dms(tao, model)
    for dm in dms:
        rep.append(
            {
            "name": dm.name,
            "class": "klystron",
            "data": dm.to_json()
            }
        )
    
    
    #quad
    dm = build_quad_dm(tao)
    rep.append(
        {
        "name": "quad",
        "class": "tabular",
        "data": dm.to_json()
        }
    )
    
    #otr2
    dm = beginning_meas_twiss_datamap('WS02', 'WIRE:IN20:561' )
    rep.append(
        {
        "name": "beginning_WS02",
        "class": "tabular",
        "data": dm.to_json()
        }
    )
    
    
    #energy_meas
    dm = build_energy_dm(model)
    rep.append(
        {
        "name": "tao_energy_measurements",
        "class": "tabular",
        "data": dm.to_json()
        }
    )
    
    return rep

In [5]:
# add to master
for model in ["cu_hxr", "cu_sxr"]:
    MASTER[model] = build_json_rep(model)

# cu_spec (i.e. the injector)

In [6]:
def build_cu_spec_linac_dm():
    dat =  [
    
        {'name': 'L0A_phase',
             'pvname':'ACCL:IN20:300:L0A_PDES', # deg
             'bmad_factor': 1/360,
             'bmad_name': 'L0A',
             'bmad_attribute': 'phi0'
            },
        {'name': 'L0B_phase',
             'pvname':'ACCL:IN20:400:L0B_PDES', # deg
             'bmad_factor': 1/360,
             'bmad_name': 'L0B',
             'bmad_attribute': 'phi0'
            },
        {'name': 'L0A_voltage',
             'pvname':'ACCL:IN20:300:L0A_ADES', # MV
             'bmad_factor': 1e6,
             'bmad_name': 'L0A',
             'bmad_attribute': 'voltage'
            },    
        {'name': 'L0B_voltage',
             'pvname':'ACCL:IN20:400:L0B_ADES', # MV
             'bmad_factor': 1e6,
             'bmad_name': 'L0B',
             'bmad_attribute': 'voltage'
            }
    ]
    df = pd.DataFrame(dat)
    dm = TabularDataMap(df, pvname='pvname', element='bmad_name', attribute='bmad_attribute', factor='bmad_factor')
    return dm


In [7]:
def build_cu_spec_json_rep():
    #set up tao
    model = 'cu_spec'
    options = '-noplot'
    init = f'-init $LCLS_LATTICE/bmad/models/{model}/tao.init {options}'
    
    tao = Tao(init)
    
    
    rep = []
    
    # BPMs
    dm = build_bpm_dm(tao, model)
    rep.append(
        {
        "name": "bpms",
        "class": "tabular",
        "data": dm.to_json()
        }
    )     
    
    # Correctors
    dm = build_corrector_dm(tao)
    rep.append(
        {
        "name": "correctors",
        "class": "tabular",
        "data": dm.to_json()
        }
    )        
    
    # quad
    dm =  build_quad_dm(tao)
    rep.append(
        {
        "name": "quad",
        "class": "tabular",
        "data": dm.to_json()
        }
    )
    
    #solenoid
    dm = build_solenoid_dm(tao, model)
    rep.append(
        {
        "name": "solenoid",
        "class": "tabular",
        "data": dm.to_json()
        }
    )      
    
    # Linac
    dm = build_cu_spec_linac_dm()
    rep.append(
        {
        "name": "linac",
        "class": "tabular",
        "data": dm.to_json()
        }
    )    
    
    return rep


# add to master
MASTER["cu_spec"] = build_cu_spec_json_rep()

# SC lines

In [8]:
## sc lines
def build_json_rep_sc(model):
    #set up tao
    # Basic model with options
    MODEL = model
    OPTIONS = f'-noplot'
    INIT = f'-init $LCLS_LATTICE/bmad/models/{MODEL}/tao.init {OPTIONS}'
    tao = Tao(INIT)
    
    
    rep = []
    
    # BPM
    dm = build_bpm_dm(tao, model)
    rep.append(
        {
        "name": "bpms",
        "class": "tabular",
        "data": dm.to_json()
        }
    )    
    
    # Cavities
    dm = build_cavity_dm(tao, model)
    rep.append(
        {
        "name": "cavities",
        "class": "tabular",
        "data": dm.to_json()
        }
    )        
    
    # Correctors
    dm = build_corrector_dm(tao)
    rep.append(
        {
        "name": "correctors",
        "class": "tabular",
        "data": dm.to_json()
        }
    )    
    
    #energy_meas
    dm = build_energy_dm(model)
    rep.append(
        {
        "name": "tao_energy_measurements",
        "class": "tabular",
        "data": dm.to_json()
        }
    )    
    
    
    #quad
    dm = build_quad_dm(tao)
    rep.append(
        {
        "name": "quad",
        "class": "tabular",
        "data": dm.to_json()
        }
    )

    #quad_corrector
    dm = build_sc_quad_corrector_dm(tao)
    rep.append(
        {
        "name": "quad_corrector",
        "class": "tabular",
        "data": dm.to_json()
        }
    )
    
    #solenoid
    dm = build_solenoid_dm(tao, model)
    rep.append(
        {
        "name": "solenoid",
        "class": "tabular",
        "data": dm.to_json()
        }
    )    
    
    
    return rep


In [9]:
# add to master
for model in [
    "sc_hxr",
    "sc_sxr",
    "sc_bsyd",
    # "sc_dasel", # this is incomplete, missing Q27 etc. device names
    "sc_inj",
    "sc_diag0",
]:
    MASTER[model] = build_json_rep_sc(model)

# Dump all to master file

In [10]:
with open(os.path.join(data_dir, 'datamaps_master.json'), "w") as f:
    json.dump(MASTER, f)

# Test

In [11]:
from lcls_live.datamaps import get_datamaps

DM = get_datamaps('sc_inj')
DM.keys()

dict_keys(['bpms', 'cavities', 'correctors', 'tao_energy_measurements', 'quad', 'quad_corrector', 'solenoid'])

In [15]:
DM['quad_corrector'].data

,bmad_name,pvname_rbv,pvname,bmad_factor,bmad_attribute
0,SQ01B,QUAD:GUNB:212:2:BACT,QUAD:GUNB:212:2:BDES,-0.1,k1l
1,CQ01B,QUAD:GUNB:212:1:BACT,QUAD:GUNB:212:1:BDES,-0.1,k1l
2,SQ02B,QUAD:GUNB:823:2:BACT,QUAD:GUNB:823:2:BDES,-0.1,k1l
3,CQ02B,QUAD:GUNB:823:1:BACT,QUAD:GUNB:823:1:BDES,-0.1,k1l


In [13]:
DM['quad'].pvlist

['QUAD:L0B:0185:BACT',
 'QUAD:HTR:120:BACT',
 'QUAD:HTR:140:BACT',
 'QUAD:HTR:300:BACT',
 'QUAD:HTR:320:BACT',
 'QUAD:HTR:365:BACT',
 'QUAD:HTR:385:BACT',
 'QUAD:HTR:440:BACT',
 'QUAD:HTR:460:BACT',
 'QUAD:HTR:830:BACT',
 'QUAD:HTR:860:BACT',
 'QUAD:HTR:960:BACT',
 'QUAD:HTR:980:BACT',
 'QUAD:COL0:240:BACT',
 'QUAD:COL0:260:BACT',
 'QUAD:COL0:280:BACT',
 'QUAD:COL0:320:BACT',
 'QUAD:COL0:400:BACT',
 'QUAD:COL0:480:BACT',
 'QUAD:COL0:560:BACT',
 'QUAD:COL0:640:BACT',
 'QUAD:COL0:720:BACT',
 'QUAD:COL0:800:BACT',
 'QUAD:COL0:880:BACT',
 'QUAD:COL0:940:BACT']

In [14]:
## Check live values
# def check_live(datamap, epics):
#     return dict(zip(datamap.pvlist, epics.caget_many(datamap.pvlist) ))
# 
# import epics
# check_live(DM['solenoid'], epics)